In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
device = "cuda"
import pickle
import torch
import numpy as np
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from pathlib import Path
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from tqdm import tqdm
from IPython.display import display, clear_output
from subprocess import check_output
import matplotlib.pyplot as plt
import imageio

In [ ]:
cfg = "00023-white_yc05_yw04-mirror-auto4-gamma10-noaug/network-snapshot-021800.pkl"
#cfg = "00030-v4-mirror-auto4-gamma100-batch64-noaug-resumecustom/network-snapshot-011289.pkl"
fm = Path(f"/data/natsuki/training116/{cfg}")
assert fm.is_file()
with open(fm, 'rb') as f:
    G = pickle.load(f)['G_ema'].to(device)
def mapping(seed=1, psi=1):
    label = torch.zeros([1, G.c_dim], device=device)
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    w = G.mapping(z, label, truncation_psi=psi)
    return w
def synthesis(w, synth=True, retarr=False):
    if synth:
        synth_image = G.synthesis(w, noise_mode='const')
    else:
        synth_image = w
    synth_image = (synth_image + 1) * (255/2)
    synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
    img = Image.fromarray(synth_image, 'RGB')
    if retarr:
        return img, synth_image
    else:
        return img

In [ ]:
seeds = [
    int(p.split("_")[1])
    for p in sorted([p.stem for p in Path(f"/data/natsuki/fact_hatsune_miku_{cfg.split('/')[0]}").glob("*.png")], reverse=True)
][:32*100]

In [ ]:
batch = 32
sample_num = len(seeds)
label = torch.zeros([1, G.c_dim], device=device)
assert sample_num % batch == 0
ws = list()
for j in tqdm(range(sample_num//batch)):
    ws += list(G.mapping(torch.from_numpy(np.array([np.random.RandomState(seeds[batch*j+i]).randn(G.z_dim) for i in range(batch)])).to(device), label).cpu().numpy()[:,0,:])
ws = np.array(ws)
print(ws.shape)

In [ ]:
W = 8; H = 8
cluster_num = W*H
km = KMeans(cluster_num, init="random")
km.fit(ws)
print(km.cluster_centers_.shape)

In [ ]:
dst = Image.new("RGB", (W*512, H*512))
for h in range(H):
    for w in range(W):
        tmp = synthesis(torch.from_numpy(km.cluster_centers_[None,None,W*h+w]).repeat((1,16,1)).to(device))
        font = ImageFont.truetype("/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf", 50)
        ImageDraw.Draw(tmp).text((0,0), f"{W*h+w}", (0,0,0), font)
        dst.paste(tmp, (512*w, 512*h))
dst

In [ ]:
ckd = list(set(map(int, "16 2 10 27 33 34 46 41 51 63".split())))
print(len(ckd), ckd)

In [ ]:
X = np.linspace(0, 1)
F = lambda X: np.tanh(5*(X-.5))*.5+.5
Y = F(X)
plt.plot(X, Y)

In [ ]:
def intp(ws, B=5):
    arrs = list()
    for i in range(len(ws)):
        for j in range(B):
            clear_output(wait=True)
            p = F(j/B)
            wm = (1-p)*ws[None,None,i] + (p)*ws[None,None,(i+1)%len(ws)]
            tmp, arr = synthesis(torch.from_numpy(wm).repeat((1,16,1)).to(device), retarr=True)
            arrs.append(arr)
            display(tmp)
    return arrs
arrs = intp(km.cluster_centers_[ckd])

In [ ]:
dname = "intp.gif"
imageio.mimsave(dname, arrs, fps=5)
print(check_output(f"du -sh {dname}", shell=True).decode(), end="")
print(check_output(f"mogrify -layers 'optimize' -fuzz 3% -loop 0 {dname}", shell=True).decode(), end="")
print(check_output(f"du -sh {dname}", shell=True).decode(), end="")